    
<h1 style="text-align: center; color: purple;" markdown="1">Econ 320: Python Assignment 4
</h1>

<h2 style="text-align: center; color: #012169" markdown="1">Wooldridge C4.11 (Modified) </h2>



## Jerry Hong

## Package setup

In [1]:
# import necessary packages
import wooldridge as woo
import numpy as np 
import pandas as pd
import matplotlib as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api  as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms # BP test
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

Use the data in HTV form wooldridge to answer this question. 

## 1. Estimate the regression model

$$ educ = \beta_0 + \beta_1 *motheduc + \beta_2 *fatheduc + \beta_3 *abil + \beta_4 *abil^2 + u$$
by OLS take a look at the results in the usual form but report the results using stargazer. 

## 2. Inference 

* 2.a Test the null hypothesis that educ is linearly related to abil against the alternative that the relationship is quadratic. Show the tstat and the pvalue associated with that test. Do you reject the null hypothesis? what does that mean? EXPLAIN (*Hint*: no need to run an extra test from the regression results you can answer this question, see the results for abil and $abil^2$) 

* 2.b Test the following hypothesis

$$ H_0: \beta_{motheduc}=0.3$$
$$ H_1:\beta_{motheduc}<0.3$$

This is a one tail test. 
**EXPLAIN your result**

* 2.c Test the following hypothesis

$$ H_0: \beta_{motheduc}=0.09$$
$$ H_1:\beta_{motheduc}>0.09$$

This is a one tail test. 
**EXPLAIN your result** 

In [2]:
# to include the term ability squared you can create a separate variable 
# or even easier use the I(function) in the ols command to add the term 

htv = woo.dataWoo('htv')
res = smf.ols(formula='educ ~ motheduc + fatheduc + abil + I(abil ** 2)', data=htv).fit()
res.summary()
st=Stargazer([res])
st.rename_covariates({"I(abil ** 2)": "abil2"})
st.covariate_order(['Intercept', 'motheduc', 'fatheduc', 'abil', 'I(abil ** 2)'])
HTML(st.render_html())

### Observations

The r-sqaured value is 0.444, so the fit is somewhat decent. The p-value for any confidence interval is less than its value, so we reject the null hypothesis suggesting the coefficient for mother education is greater than 0.3.

In [6]:
# manually confirm the formulas, i.e. extract coefficients and SE:
b = res.params
se = res.bse

print("T-test for linear vs quadratic relation on ability")
# reproduce t statistic:
tstat = b[4]/se[4]

print(f'tstat: \n{tstat}\n')

# reproduce p value:
pval = 2 * stats.t.cdf(-abs(tstat), 1225)

# use the np.around() function to round array 
print(f'pval: \n{np.around(pval,4)}\n')

print("T-test for mother educ < 0.3")
# reproduce t statistic:
tstat1 = b[1] - 0.3/se[1]

print(f'tstat: \n{tstat1}\n')

# reproduce p value:
pval1 = 2 * stats.t.cdf(-abs(tstat1), 1225)

# use the np.around() function to round array 
print(f'pval: \n{np.around(pval1, 4)}\n')


T-test for linear vs quadratic relation on ability
tstat: 
6.093400049308947

pval: 
0.0

T-test for mother educ < 0.3
tstat: 
-10.487663076513327

pval: 
0.0



<br>

## 3.  Using the equation in part (2), test $H_0: \beta_1=\beta_2$ against a two-sided alternative. What is the p-value of the test? 

Remember this requires for creating a new regression with a $\theta_1=\beta_1-\beta_2$ as shown in your book and then test for $H_0: \theta_1=0$

 Let's change the regression to create $\theta_1=\beta_1-\beta_2$ 

Add and subrstact $\beta_2 motheduc$ and create a variable $parentedu=motheduc+fatheduc$ see below: 

$$ educ = \beta_0 + \beta_1 motheduc - \beta_2 motheduc + \beta_2 motheduc+ \beta_2 fatheduc + \beta_3 abil + \beta_4 abil^2 + u$$

$$ educ = \beta_0 + (\beta_1 - \beta_2)   motheduc + \beta_2  (motheduc+fatheduc) + \beta_3 abil + \beta_4 abil^2 + u$$
$$ educ = \beta_0 + \theta_1   motheduc + \beta_2  (parentedu) + \beta_3 abil + \beta_4 abil^2 + u$$

By testing the null hypothesis that $H_0:\theta_1=0$ with $alpha=0.05$ we are testing $H_0: \beta_1=\beta_2$ So we just run the regression that has $\theta_1$ as a regressor and look at the t-test for $\theta_1$

In [13]:
#critival Values for alpha=5% and 1%
print("critical values for alpha 5% and 1% 2 tails")
cv_t = stats.t.ppf(1-0.05/2, 1225)

print(f'cv_t: {cv_t}\n')

# CV for alpha=5% and 1% using the normal approximation:
cv_n = stats.norm.ppf(0.05)

print(f'cv_n: {cv_n}\n')

#qt(1-alpha_2, 1225)

# create parenteduc
htv['parenteduc'] = htv['motheduc'] + htv['fatheduc']
# regression with theta1
res1 = smf.ols(formula = 'educ ~ motheduc + parenteduc + abil + I(abil **2)', data = htv).fit()

st=Stargazer([res1])
st.rename_covariates({"I(abil ** 2)": "abil2"})
st.covariate_order(['motheduc' , 'parenteduc' , 'abil', 'I(abil ** 2)', 'Intercept'])
HTML(st.render_html())


critical values for alpha 5% and 1% 2 tails
cv_t: 1.961902414756263

cv_n: -1.6448536269514729




*** 
#### Complete the empty spaces

The value of $\theta_1$ is equal to -0.028 with a t-stat of 244.906 and a p-value of 0.003 this means that we reject the null hypothesis that  $H_0:\theta_1=0$ which means that $\beta_1$ does not $\beta_2$ therefore the level of education of mother's and father's education has high magnitute. (this might be completely wrong, but this is my best approximation)

***
<br>

## 4. 	Add the two college tuition variables to the regression from part (2) and determine whether they are jointly statistically significant. 
First, do the F-test step-by-step

In [14]:
## F test step by step
n = htv.shape[0]
# Unrestricted OLS regression:  
reg_ur= smf.ols(formula = 'educ ~ motheduc + parenteduc + abil + I(abil **2)+ tuit17 + tuit18', data = htv)
fit_ur= reg_ur.fit()
r2_ur= fit_ur.rsquared

print(f'r2_ur: {r2_ur}\n')

# Restricted OLS regression:
reg_r= smf.ols(formula = 'educ ~ motheduc + parenteduc + abil + I(abil **2)', data = htv)
fit_r= reg_r.fit()
r2_r= fit_r.rsquared


print(f'r2_r: {r2_r}\n')

# F statistic:
fstat = (r2_ur - r2_r)/(1-r2_ur)* (n-6)/ 3

print(f'fstat: {fstat}\n')

# CV for alpha=1% using the F distribution with 3 and 1223 d.f. :
cv = stats.f.ppf(1-0.01, 3, 1230)

print(f'cv: {cv}\n')


r2_ur: 0.44511171101427116

r2_r: 0.4443500858908753

fstat: 0.5600101074641594

cv: 3.7975666835811324




#### Then use any of the other methods


In [15]:
# F test using compare_f_test
fit_ur.compare_f_test(fit_r)



(0.8393288742998524, 0.4322490378049605, 2.0)

<br> 
This shows that in this case we fail to reject the null hypothesis**, that the coefficients are jointly zero. 

## 5.  Use function `.conf_int()` to find the confidence intervals of all the parameters in the unsrestricted model from (4) What do you conclude? EXPLAIN this results in the light of the significance of your coeficients

In [16]:
# find confidence intervals
fit_ur.conf_int(alpha=0.05)



,0,1
Intercept,7.468242,8.695487
motheduc,0.001994,0.166906
parenteduc,0.069954,0.146931
abil,0.339500,0.458585
I(abil ** 2),0.034246,0.066863
tuit17,-0.106866,0.138384
tuit18,-0.124810,0.124931


The confidence intervals of these parameters show that the unrestricted model has high variation given we are adding more variables to the model.

&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">ECON220 Python Programming Laboratory</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px">Professor <em> Paloma Lopez de mesa Moyano</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>paloma.moyano@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center;font-size: 15px">Department of Economics</a></p>
<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;